In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [2]:
load_dotenv()

model = ChatGroq(
    model='llama-3.3-70b-versatile',
    temperature=0.3
)

In [3]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/3315923e0c0a186fa08b4de4/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()


In [4]:

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [5]:
model_with_tools = model.bind_tools([get_conversion_factor, convert])

In [6]:
res = model_with_tools.invoke("what is the conversion factor of USD and PKR, and can you convert 10 usd to pkr")

In [7]:
messages = [HumanMessage("what is the conversion factor of USD and PKR, and can you convert 10 usd to pkr")]

In [8]:
messages.append(res)

In [9]:
messages

[HumanMessage(content='what is the conversion factor of USD and PKR, and can you convert 10 usd to pkr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nmy88c6r3', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': '9kypca2zp', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 347, 'total_tokens': 385, 'completion_time': 0.065738836, 'prompt_time': 0.047521595, 'queue_time': 0.050103635, 'total_time': 0.113260431}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2f3a824c-7186-4519-bbfc-d0f7b46dba3a-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency

In [10]:
import json

for tool in res.tool_calls:
    if tool['name'] == 'get_conversion_factor':
        tool_message = get_conversion_factor.invoke(tool)
        conversion_rate = json.loads(tool_message.content)['conversion_rate']
        messages.append(tool_message)
        
    if tool['name'] == "convert":
        tool['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool)
        messages.append(tool_message2)

In [11]:
messages

[HumanMessage(content='what is the conversion factor of USD and PKR, and can you convert 10 usd to pkr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nmy88c6r3', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': '9kypca2zp', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 347, 'total_tokens': 385, 'completion_time': 0.065738836, 'prompt_time': 0.047521595, 'queue_time': 0.050103635, 'total_time': 0.113260431}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2f3a824c-7186-4519-bbfc-d0f7b46dba3a-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency

In [13]:
output = model_with_tools.invoke(messages)

In [15]:
print(output.content)

The conversion factor of USD to PKR is 283.637. 

10 USD is equivalent to 2836.37 PKR.
